참고 코드 : https://wikidocs.net/45101

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)
print(tokenizer.word_index)

단어 집합의 크기 : 1346
{'있었다': 1, '소년은': 2, '소녀가': 3, '한': 4, '소녀의': 5, '있는': 6, '그러나': 7, '소녀는': 8, '소년이': 9, '했다': 10, '않았다': 11, '가': 12, '있다': 13, '앉아': 14, '안고': 15, '더': 16, '소년의': 17, '전에': 18, '그': 19, '걸': 20, '수': 21, '좀': 22, '뵈지': 23, '하는': 24, '같은': 25, '손을': 26, '하고': 27, '개울가로': 28, '나왔다': 29, '이': 30, '그리고는': 31, '가을': 32, '것이었다': 33, '없이': 34, '것': 35, '번': 36, '이게': 37, '”소녀가': 38, '”소년은': 39, '윤': 40, '그런데': 41, '개울': 42, '다음': 43, '그냥': 44, '그대로': 45, '그러다가': 46, '바보': 47, '저도': 48, '모르게': 49, '소녀': 50, '같았다': 51, '날': 52, '얼굴이': 53, '몇': 54, '시작했다': 55, '소리가': 56, '무슨': 57, '저': 58, '못': 59, '꽃을': 60, '우리': 61, '많이': 62, '게': 63, '그날': 64, '호두': 65, '소녀를': 66, '물장난을': 67, '것이다': 68, '서울': 69, '이런': 70, '듯이': 71, '며칠째': 72, '학교에서': 73, '돌아오는': 74, '오늘은': 75, '징검다리': 76, '뚝에': 77, '길을': 78, '날은': 79, '세수를': 80, '분홍': 81, '스웨터': 82, '이번에는': 83, '낸다': 84, '자꾸': 85, '하나': 86, '일어나': 87, '징검다리를': 88, '뛰어': 89, '다': 90, '생각됐다': 91, '않는다': 92, '갈꽃이': 93, '것만': 94, '서': 95, '조약돌을

In [ ]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 1702


In [ ]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print(max_len)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

27


In [ ]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [ ]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
54/54 [==============================] - 3s 14ms/step - loss: 7.2075 - accuracy: 0.0082
Epoch 2/200
54/54 [==============================] - 1s 15ms/step - loss: 7.1867 - accuracy: 0.0118
Epoch 3/200
54/54 [==============================] - 1s 15ms/step - loss: 7.1315 - accuracy: 0.0141
Epoch 4/200
54/54 [==============================] - 1s 14ms/step - loss: 6.9741 - accuracy: 0.0141
Epoch 5/200
54/54 [==============================] - 1s 14ms/step - loss: 6.9268 - accuracy: 0.0141
Epoch 6/200
54/54 [==============================] - 1s 14ms/step - loss: 6.9089 - accuracy: 0.0141
Epoch 7/200
54/54 [==============================] - 1s 14ms/step - loss: 6.8879 - accuracy: 0.0141
Epoch 8/200
54/54 [==============================] - 1s 14ms/step - loss: 6.8567 - accuracy: 0.0141
Epoch 9/200
54/54 [==============================] - 1s 14ms/step - loss: 6.8164 - accuracy: 0.0141
Epoch 10/200
54/54 [==============================] - 1s 14ms/step - loss: 6.7622 - accuracy: 0.0141

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

소년은 저도 모르게 주머니 속 호두 알을 만지작거리며 한 손으로는 수없이 갈꽃을 휘어 꺾고 있었다 대로 잠자코 있었다 대로 잠자코 있었다


In [ ]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [ ]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 소녀는 학교에서 잠그고 물장난을 하던


In [ ]:
print(sentence_generation(model, tokenizer, '소년은', 20))

소년은 저도 모르게 주머니 속 호두 알을 만지작거리며 한 손으로는 수없이 갈꽃을 휘어 꺾고 있었다 대로 잠자코 있었다 대로 잠자코 있었다


In [ ]:
print(sentence_generation(model, tokenizer, '소년은 공연히', 20))

소년은 공연히 열적어 책 보를 집어던지고는 외양간으로 가 쇠잔 시작했다 있다 가에 앉아 있는 것이다 들어서 것이다 들어서 것이다 들어서 가 서울서


In [ ]:
print(sentence_generation(model, tokenizer, '그 날 밤', 20))

그 날 밤 가을 하늘이 언제 그랬는가 싶게 구름 한 점 움켜 낸다 개어 있었다 쪽빛으로 수없이 앉아 있었다 대로 잠자코 있었다 수없이


In [ ]:
import pandas as pd

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
text = open('drive/MyDrive/colab_test/소나기.txt', 'r', encoding='UTF-8')
lines = text.readlines()

In [ ]:
print(lines[4])

다음 날은 좀 늦게 개울가로 나왔다.



In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
from hanspell import spell_checker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-5onqikiy
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-5onqikiy
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=6c8d85687510872de4ebb5e21395fae4648cd45aff19cfa181d6dbf789a75b58
  Stored in directory: /tmp/pip-ephem-wheel-cache-llqm4ul0/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
for i in range(len(lines)):
  text_spell = spell_checker.check(lines[i])
  lines[i] = text_spell.checked

KeyboardInterrupt: ignored

In [ ]:
print(lines[0])

소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女) 딸이라는 걸 알 수 있었다. 


In [ ]:
text = ''.join(lines)

In [ ]:
text = text.replace('”', '')
text = text.replace('‘','')
text = text.replace('“','')

In [ ]:
text_list = text.split('.')
print(text_list[0])
text = '\n'.join(text_list)

소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女) 딸이라는 걸 알 수 있었다


In [ ]:
print(text)

소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女) 딸이라는 걸 알 수 있었다
 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다
 서울 서는 이런 개울물을 보지 못하기나 한 듯이
 벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다
 그런데, 어제까지 개울 기슭에서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다
 소년은 개울 뚝에 앉아 버렸다
 소녀가 비키기를 기다리자는 것이다
 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다
다음 날은 좀 늦게 개울가로 나왔다
이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다
 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다
한참 세수를 하고 나더니, 이번에는 물속을 빤히 들여다본다
 얼굴이라도 비추어 보는 것이리라
 갑자기 물을 움켜 낸다
 고기 새끼라도 지나가는 듯
소녀는 소년이 개울 뚝에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다
 그러나, 번번이 허탕이다
 그대로 재미있는 양, 자꾸 물만 움킨다
 어제처럼 개울을 건너는 사람이 있어야 길을 비킬 모양이다
그러다가 소녀가 물속에서 무엇을 하나 집어 낸다
 하얀 조약돌이었다
 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다
다 건너가더니만 홱 이리로 돌아서며,이 바보
조약돌이 날아왔다
소년은 저도 모르게 벌떡 일어섰다
단발머리를 나풀거리며 소녀가 막 달린다
 갈밭 사잇길로 들어섰다
 뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐
이제 저쯤 갈밭머리로 소녀가 나타나리라
 꽤 오랜 시간이 지났다고 생각됐다
 그런데도 소녀는 나타나지 않는다
 발돋움을 했다
 그러고도 상당한 시간이 지났다고 생각됐다
저쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다
 소녀가 갈꽃을 안고 있었다
 그리고, 이제는 천천한 걸음이었다
 유난히 맑은 가을 햇살이 소녀의 갈꽃 머리에서 반짝거렸다
 소녀 아닌 갈꽃이 들길을 걸어가는 것만 같았다
소년은 이 갈꽃이 아주 뵈지 않게 되기까지 그대로 서 있었다
 문득, 소녀 가던지 조약돌을 내려다보았다
 물

Char-RNN : https://jin-z.tistory.com/5

주의 : tensorflow 재설치를 통해 버전을 변경함

In [ ]:
!git clone https://github.com/solaris33/char-rnn-tensorflow.git

Cloning into 'char-rnn-tensorflow'...
remote: Enumerating objects: 37, done.
remote: Total 37 (delta 0), reused 0 (delta 0), pack-reused 37
Unpacking objects: 100% (37/37), done.


In [1]:
import tensorflow as tf

tf.__version__

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

'1.14.0'

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.14 

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? 
Your response ('') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 41 kB/s 
     |████████████████████████████████| 3.1 MB 46.4 MB/s 
     |████████████████████████████████| 488 kB 62.2

In [4]:
cd char-rnn-tensorflow

/content/char-rnn-tensorflow


In [5]:
!python train.py

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [31]:
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np


class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        # 전처리된 파일들("vocab.pkl", "data.npy")이 이미 존재하면 이를 불러오고 없으면 데이터 전처리를 진행합니다.
        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        # 배치를 생성하고 배치 포인터를 배치의 시작지점으로 리셋합니다.
        self.create_batches()
        self.reset_batch_pointer()

    # 데이터 전처리를 진행합니다.
    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        # 데이터에서 문자(character)별 등장횟수를 셉니다.
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs) # 전체 문자들(Chracters)
        self.vocab_size = len(self.chars) # 전체 문자(단어) 개수
        self.vocab = dict(zip(self.chars, range(len(self.chars)))) # 단어들을 (charcter, id) 형태의 dictionary로 만듭니다.
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        # 데이터의 각각의 character들을 id로 변경합니다.
        self.tensor = np.array(list(map(self.vocab.get, data)))
        # id로 변경한 데이터를 "data.npy" binary numpy 파일로 저장힙니다.
        np.save(tensor_file, self.tensor)

    # 전처리한 데이터가 파일로 저장되어 있다면 파일로부터 전처리된 정보들을 읽어옵니다.
    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    # 전체 데이터를 배치 단위로 묶습니다.
    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # 데이터 양이 너무 적어서 1개의 배치도 만들수없을 경우, 에러 메세지를 출력합니다.
        if self.num_batches == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        # 배치에 필요한 정수만큼의 데이터만을 불러옵니다. e.g. 1115394 -> 1115000
        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        # 타겟 데이터는 인풋 데이터를 한칸 뒤로 민 형태로 구성합니다.
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        # batch_size 크기의 배치를 num_batches 개수 만큼 생성합니다. 
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)

    # 다음 배치롤 불러오고 배치 포인터를 1만큼 증가시킵니다.  
    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    # 배치의 시작점을 데이터의 시작지점으로 리셋합니다.
    def reset_batch_pointer(self):
        self.pointer = 0

In [51]:
ls

char-rnn-tensorflow/  drive/  sample_data/


In [52]:
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

from utils import TextLoader
tf.reset_default_graph()
# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'drive/MyDrive/colab_test/sonagi' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
#data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 200
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

loading preprocessed files
0(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 7.166
1(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 6.227
2(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 5.883
3(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 5.719
4(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 5.502
5(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 5.332
6(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 3, 손실함수(loss): 5.322
7(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 3, 손실함수(loss): 5.153
8(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 3, 손실함수(loss): 5.013
9(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 4, 손실함수(loss): 5.059
10(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 4, 손실함수(loss): 4.912
11(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 4, 손실함수(loss): 4.784
12(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 4.857
13(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 4.716
14(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 4.569
15(학습한 배치개수)/600(학습할 배치개수), 반복(epoch): 6, 손실함수(loss): 4.663
16(학습한 배치개수)/600(학습할 배치